### Proposition 01 (Ch4): Order with the maximum OrderID
**Concept:** Self-contained scalar subquery in WHERE  
**Why:** Finds the single row whose OrderID equals the overall MAX(OrderID).

In [ ]:
SELECT o.OrderID, o.CustomerID, o.OrderDate
FROM Sales.Orders AS o
WHERE o.OrderID = (SELECT MAX(OrderID) FROM Sales.Orders);

### Proposition 02 (Ch4): Customers who placed orders in 2016
**Concept:** Self-contained multivalued subquery with IN  
**Why:** Filters customers to those appearing in the 2016 Orders list.

In [ ]:
SELECT c.CustomerID, c.CustomerName
FROM Sales.Customers AS c
WHERE c.CustomerID IN (
    SELECT DISTINCT o.CustomerID
    FROM Sales.Orders AS o
    WHERE o.OrderDate >= '2016-01-01'
      AND o.OrderDate <  '2017-01-01'
)
ORDER BY c.CustomerName;

### Proposition 03 (Ch4): Latest order date per customer (correlated subquery)
**Concept:** Correlated scalar subquery in SELECT  
**Why:** For each customer, compute MAX(OrderDate) from Orders for that customer.

In [ ]:
SELECT 
    c.CustomerID,
    c.CustomerName,
    (SELECT MAX(o.OrderDate)
     FROM Sales.Orders AS o
     WHERE o.CustomerID = c.CustomerID) AS LatestOrderDate
FROM Sales.Customers AS c
ORDER BY LatestOrderDate DESC, c.CustomerID;

### Proposition 04 (Ch4): Customers who ever ordered a chilled (refrigerated) item
**Concept:** EXISTS with correlated subquery  
**Why:** Checks if there exists any order line for the customer with a chiller stock item.

In [ ]:
SELECT c.CustomerID, c.CustomerName
FROM Sales.Customers AS c
WHERE EXISTS (
    SELECT 1
    FROM Sales.Orders AS o
    JOIN Sales.OrderLines AS ol ON ol.OrderID = o.OrderID
    JOIN Warehouse.StockItems AS si ON si.StockItemID = ol.StockItemID
    WHERE o.CustomerID = c.CustomerID
      AND si.IsChillerStock = 1
)
ORDER BY c.CustomerID;

### Proposition 05 (Ch4): Previous order date per order (same customer)
**Concept:** Correlated subquery returning “previous” value  
**Why:** For each order, look up the most recent earlier OrderDate for the same customer.

In [ ]:
SELECT 
    o.OrderID,
    o.CustomerID,
    o.OrderDate,
    (SELECT MAX(o2.OrderDate)
     FROM Sales.Orders AS o2
     WHERE o2.CustomerID = o.CustomerID
       AND o2.OrderDate < o.OrderDate) AS PreviousOrderDate
FROM Sales.Orders AS o
ORDER BY o.CustomerID, o.OrderDate;

### Proposition 06 (Ch5): Top 10 orders by computed order total (derived table)
**Concept:** Derived table in FROM with GROUP BY  
**Why:** Compute order totals from OrderLines, then select the largest totals.

In [ ]:
SELECT TOP (10) t.OrderID, t.OrderTotal
FROM (
    SELECT ol.OrderID, SUM(ol.Quantity * ol.UnitPrice) AS OrderTotal
    FROM Sales.OrderLines AS ol
    GROUP BY ol.OrderID
) AS t
ORDER BY t.OrderTotal DESC, t.OrderID;

### Proposition 07 (Ch5): Top 10 customers by 2016 invoice revenue (CTE)
**Concept:** CTE + GROUP BY  
**Why:** Stage 2016 invoice lines, then sum Quantity*UnitPrice per customer.

In [ ]:
WITH Lines2016 AS (
    SELECT i.CustomerID, il.Quantity, il.UnitPrice
    FROM Sales.Invoices AS i
    JOIN Sales.InvoiceLines AS il ON il.InvoiceID = i.InvoiceID
    WHERE i.InvoiceDate >= '2016-01-01' AND i.InvoiceDate < '2017-01-01'
),
CustTotals AS (
    SELECT CustomerID, SUM(Quantity * UnitPrice) AS Revenue2016
    FROM Lines2016
    GROUP BY CustomerID
)
SELECT TOP (10) c.CustomerID, c.CustomerName, ct.Revenue2016
FROM CustTotals AS ct
JOIN Sales.Customers AS c ON c.CustomerID = ct.CustomerID
ORDER BY ct.Revenue2016 DESC, c.CustomerID;

### Proposition 08 (Ch5): Customers with > 10 orders (CTE, multiple references)
**Concept:** CTE + GROUP BY + HAVING  
**Why:** Precompute order counts, then filter and join to customer names.

In [ ]:
WITH OrderCounts AS (
    SELECT o.CustomerID, COUNT(*) AS NumOrders
    FROM Sales.Orders AS o
    GROUP BY o.CustomerID
)
SELECT oc.CustomerID, c.CustomerName, oc.NumOrders
FROM OrderCounts AS oc
JOIN Sales.Customers AS c ON c.CustomerID = oc.CustomerID
WHERE oc.NumOrders > 10
ORDER BY oc.NumOrders DESC, oc.CustomerID;

### Proposition 09 (Ch5): Top 3 most recent orders per customer (CROSS APPLY)
**Concept:** CROSS APPLY with a top-N per outer row  
**Why:** Apply a “top 3 recent orders” subquery to each customer.

In [ ]:
SELECT 
    c.CustomerID,
    c.CustomerName,
    r.OrderID,
    r.OrderDate
FROM Sales.Customers AS c
CROSS APPLY (
    SELECT TOP (3) o.OrderID, o.OrderDate
    FROM Sales.Orders AS o
    WHERE o.CustomerID = c.CustomerID
    ORDER BY o.OrderDate DESC, o.OrderID DESC
) AS r
ORDER BY c.CustomerID, r.OrderDate DESC;

### Proposition 10 (Ch5): Cities ranked by order count (derived table)
**Concept:** Derived table + GROUP BY  
**Why:** Build a city→order count list (via Customers), then present ranked results.

In [ ]:
SELECT ct.CityID, ct.CityName, ct.NumOrders
FROM (
    SELECT city.CityID, city.CityName, COUNT(*) AS NumOrders
    FROM Sales.Orders AS o
    JOIN Sales.Customers AS cust ON cust.CustomerID = o.CustomerID
    JOIN Application.Cities AS city ON city.CityID = cust.DeliveryCityID
    GROUP BY city.CityID, city.CityName
) AS ct
ORDER BY ct.NumOrders DESC, ct.CityName;